In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 256
epochs = 100
latent_dim = 256
num_samples = 1000

Loading input and target texts.

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines_input = open('./data/OpenSubtitles2016.en-nl.nl').read().split('\n')
lines_target = open('./data/OpenSubtitles2016.en-nl.en').read().split('\n')

In [4]:
for line in lines_input[: min(num_samples, len(lines_input) - 1)]:
    input_texts.append(line)
    for char in line:
        if char not in input_characters:
            input_characters.add(char)
            
for line in lines_target[: min(num_samples, len(lines_target) - 1)]:
    target_text = '\t' + line + '\n'
    target_texts.append(target_text)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of target samples: ', len(target_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of target samples:  1000
Number of unique input tokens: 79
Number of unique output tokens: 83
Max sequence length for inputs: 262
Max sequence length for outputs: 246


In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [8]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s1000samples.h5')


Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 3s - loss: 1.0964 - val_loss: 0.8519
Epoch 2/100
800/800 [==============================] - 2s - loss: 1.0275 - val_loss: 0.8474
Epoch 3/100
800/800 [==============================] - 2s - loss: 1.0231 - val_loss: 0.8443
Epoch 4/100
800/800 [==============================] - 2s - loss: 1.0195 - val_loss: 0.8448
Epoch 5/100
800/800 [==============================] - 2s - loss: 1.0196 - val_loss: 0.8437
Epoch 6/100
800/800 [==============================] - 2s - loss: 1.0177 - val_loss: 0.8383
Epoch 7/100
800/800 [==============================] - 2s - loss: 1.0137 - val_loss: 0.8458
Epoch 8/100
800/800 [==============================] - 2s - loss: 1.0171 - val_loss: 0.8334
Epoch 9/100
800/800 [==============================] - 2s - loss: 1.0073 - val_loss: 0.8335
Epoch 10/100
800/800 [==============================] - 2s - loss: 1.0073 - val_loss: 0.8323
Epoch 11/100
800/800 [===========

800/800 [==============================] - 2s - loss: 0.7321 - val_loss: 0.6264
Epoch 89/100
800/800 [==============================] - 2s - loss: 0.7301 - val_loss: 0.6282
Epoch 90/100
800/800 [==============================] - 2s - loss: 0.7284 - val_loss: 0.6252
Epoch 91/100
800/800 [==============================] - 2s - loss: 0.7257 - val_loss: 0.6238
Epoch 92/100
800/800 [==============================] - 2s - loss: 0.7218 - val_loss: 0.6251
Epoch 93/100
800/800 [==============================] - 3s - loss: 0.7234 - val_loss: 0.6217
Epoch 94/100
800/800 [==============================] - 2s - loss: 0.7224 - val_loss: 0.6282
Epoch 95/100
800/800 [==============================] - 2s - loss: 0.7207 - val_loss: 0.6164
Epoch 96/100
800/800 [==============================] - 3s - loss: 0.7147 - val_loss: 0.6306
Epoch 97/100
800/800 [==============================] - 2s - loss: 0.7213 - val_loss: 0.6218
Epoch 98/100
800/800 [==============================] - 2s - loss: 0.7138 - val_los

In [11]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
(i, char) for char, i in target_token_index.items())

In [12]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [13]:
for seq_index in range(20):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Kijk om je heen.
Decoded sentence: The s and the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the sta
-
Input sentence: Alles beweegt, alles draait.
Decoded sentence: The s and the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the sta
-
Input sentence: Zo komen wij ter wereld.
Decoded sentence: The s and the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the stare the sta
-
Input sentence: De zon, de maan, de planeten en de sterren kijken toe.
Deco